In [1]:
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path
sys.path.append(os.path.dirname(Path().resolve()))
from modules.movie_lens import MovieLens1MDataset 
from modules.factorization_machine import FactorizationMachines

/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']
  warnings.warn(f"file system 

In [2]:
dataset = MovieLens1MDataset()

In [3]:
dataset.ratings

,user_id,movie_id,rating,timestamp,user_index,gender,age,occupation,zip,rating_movie_num,...,year,rating_user_num,mean_movie_rating,min_movie_rating,max_movie_rating,1_rated_user_num,2_rated_user_num,3_rated_user_num,4_rated_user_num,5_rated_user_num
0,1,1193,5,978300760,0,F,1,10,48067,53,...,1975,1725,4.390725,1,5,12.0,33.0,161.0,582.0,937.0
1,2,1193,5,978298413,1,M,56,16,70072,129,...,1975,1725,4.390725,1,5,12.0,33.0,161.0,582.0,937.0
2,12,1193,4,978220179,11,M,25,12,32793,23,...,1975,1725,4.390725,1,5,12.0,33.0,161.0,582.0,937.0
3,15,1193,4,978199279,14,M,25,7,22903,201,...,1975,1725,4.390725,1,5,12.0,33.0,161.0,582.0,937.0
4,17,1193,5,978158471,16,M,50,1,95350,211,...,1975,1725,4.390725,1,5,12.0,33.0,161.0,582.0,937.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,5948,M,18,17,47901,181,...,1998,2,4.000000,3,5,NaN,NaN,1.0,NaN,1.0
1000205,5675,2703,3,976029116,5674,M,35,14,30030,586,...,1998,1,3.000000,3,3,NaN,NaN,1.0,NaN,NaN
1000206,5780,2845,1,958153068,5779,M,18,17,92886,259,...,1999,1,1.000000,1,1,1.0,NaN,NaN,NaN,NaN
1000207,5851,3607,5,957756608,5850,F,18,20,55410,162,...,1973,1,5.000000,5,5,NaN,NaN,NaN,NaN,1.0


In [4]:
explanatory_variable_columns_info = {
    'user_id': {'value_type': 'categorical'},
    'gender': {'value_type': 'categorical'},
    'age': {'value_type': 'categorical'},
    'zip': {'value_type': 'categorical'},
    'rating_movie_num': {'value_type': 'categorical'},
    'movie_id': {'value_type': 'categorical'},
    'year': {'value_type': 'categorical'},
    'genre': {'value_type': 'categorical', 'max_values_num': 3},
}

In [5]:
# def fetch_rate(x):
#     if 'Drama' in x.genre:
#         return 5 if 'F' == x.gender else 1
#     else:
#         return x.rating

def fetch_rate(x):
    return 1 if 'Drama' in x.genre else x.rating

# 明示的にDramaジャンルの評価を下げてみる
dataset.ratings['rating'] = dataset.ratings.apply(fetch_rate, axis=1)

In [6]:
dataset.ratings[list(explanatory_variable_columns_info.keys()) + ['rating']]

,user_id,gender,age,zip,rating_movie_num,movie_id,year,genre,rating
0,1,F,1,48067,53,1193,1975,[Drama],1
1,2,M,56,70072,129,1193,1975,[Drama],1
2,12,M,25,32793,23,1193,1975,[Drama],1
3,15,M,25,22903,201,1193,1975,[Drama],1
4,17,M,50,95350,211,1193,1975,[Drama],1
...,...,...,...,...,...,...,...,...,...
1000204,5949,M,18,47901,181,2198,1998,[Documentary],5
1000205,5675,M,35,30030,586,2703,1998,[Drama],1
1000206,5780,M,18,92886,259,2845,1999,[Drama],1
1000207,5851,F,18,55410,162,3607,1973,"[Comedy, Drama, Western]",1


In [7]:
fm = FactorizationMachines()
fm.train_model(
    datasets=dataset.ratings,
    target_variable_column='rating',
    predict_type='regression',
    epochs=100,
    explanatory_variable_columns_info=explanatory_variable_columns_info
)

Epoch 1/100
801/801 [==============================] - 3s 3ms/step - loss: 0.8947 - root_mean_squared_error: 0.9459 - val_loss: 0.7407 - val_root_mean_squared_error: 0.8606
Epoch 2/100
801/801 [==============================] - 2s 2ms/step - loss: 0.6780 - root_mean_squared_error: 0.8234 - val_loss: 0.6270 - val_root_mean_squared_error: 0.7918
Epoch 3/100
801/801 [==============================] - 3s 3ms/step - loss: 0.6025 - root_mean_squared_error: 0.7762 - val_loss: 0.5898 - val_root_mean_squared_error: 0.7680
Epoch 4/100
801/801 [==============================] - 2s 3ms/step - loss: 0.5736 - root_mean_squared_error: 0.7573 - val_loss: 0.5684 - val_root_mean_squared_error: 0.7540
Epoch 5/100
801/801 [==============================] - 2s 3ms/step - loss: 0.5582 - root_mean_squared_error: 0.7472 - val_loss: 0.5580 - val_root_mean_squared_error: 0.7470
Epoch 6/100
801/801 [==============================] - 2s 3ms/step - loss: 0.5483 - root_mean_squared_error: 0.7405 - val_loss: 0.5544 

801/801 [==============================] - 2s 3ms/step - loss: 0.4813 - root_mean_squared_error: 0.6937 - val_loss: 0.5216 - val_root_mean_squared_error: 0.7222
Epoch 49/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4807 - root_mean_squared_error: 0.6933 - val_loss: 0.5218 - val_root_mean_squared_error: 0.7224
Epoch 50/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4798 - root_mean_squared_error: 0.6927 - val_loss: 0.5223 - val_root_mean_squared_error: 0.7227
Epoch 51/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4791 - root_mean_squared_error: 0.6922 - val_loss: 0.5208 - val_root_mean_squared_error: 0.7217
Epoch 52/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4788 - root_mean_squared_error: 0.6920 - val_loss: 0.5232 - val_root_mean_squared_error: 0.7233
Epoch 53/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4780 - root_mean_squared_error: 0.6914 - val_loss: 0.5218 - val_r

801/801 [==============================] - 2s 3ms/step - loss: 0.4606 - root_mean_squared_error: 0.6787 - val_loss: 0.5187 - val_root_mean_squared_error: 0.7202
Epoch 96/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4602 - root_mean_squared_error: 0.6784 - val_loss: 0.5194 - val_root_mean_squared_error: 0.7207
Epoch 97/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4599 - root_mean_squared_error: 0.6782 - val_loss: 0.5177 - val_root_mean_squared_error: 0.7195
Epoch 98/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4596 - root_mean_squared_error: 0.6779 - val_loss: 0.5168 - val_root_mean_squared_error: 0.7189
Epoch 99/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4593 - root_mean_squared_error: 0.6777 - val_loss: 0.5189 - val_root_mean_squared_error: 0.7204
Epoch 100/100
801/801 [==============================] - 2s 3ms/step - loss: 0.4589 - root_mean_squared_error: 0.6774 - val_loss: 0.5174 - val_

In [8]:
fm.predict(datasets=dataset.ratings.iloc[100:101])

array([[1.0459383]], dtype=float32)

In [9]:
embeddings = fm.fetch_interaction_embeddings(column_name='user_id')

In [10]:
target_column = 'genre'
data = []
for voc in fm.fields_map[target_column].vocabulary:
    linear_param = fm.fetch_linear_param(column_name=target_column, value=voc)[0]
    target_embedding = fm.fetch_interaction_embedding(column_name=target_column, value=voc)
    results = []
    for i, e in enumerate(embeddings):
        results.append(np.dot(e, target_embedding))

    posi_cnt = len(np.where(np.array(results) > 0)[0])
    nega_cnt = len(np.where(np.array(results) <= 0)[0])
    posi_ratio = posi_cnt / len(results)
    data.append([voc, linear_param, posi_ratio, posi_cnt, nega_cnt, min(results), max(results), sum(results) / len(results)])

pd.DataFrame(data, columns=['value', 'linear_param', 'posi_ratio', 'posi_cnt', 'nega_cnt', 'min_dot', 'max_dot', 'avg_dot'])

,value,linear_param,posi_ratio,posi_cnt,nega_cnt,min_dot,max_dot,avg_dot
0,Documentary,1.140768,0.515643,3115,2926,-0.444434,0.426879,-0.000421
1,Sci-Fi,0.145037,0.516967,3123,2918,-0.466730,0.452191,-0.000375
2,Thriller,0.359077,0.512498,3096,2945,-0.448221,0.433663,-0.000226
3,Romance,0.461896,0.514153,3106,2935,-0.442121,0.383571,-0.000258
4,Children's,-0.128151,0.511670,3091,2950,-0.678564,0.504048,-0.000764
5,Western,0.497134,0.511670,3091,2950,-0.448613,0.213185,-0.000593
6,Film-Noir,0.752764,0.513491,3102,2939,-0.348187,0.218803,-0.000198
7,Comedy,0.430605,0.511339,3089,2952,-0.766323,0.398100,-0.000128
8,Fantasy,0.374928,0.509353,3077,2964,-0.466269,0.358939,-0.000514
9,Animation,0.508060,0.520278,3143,2898,-0.676081,0.570986,-0.000286


Dramaのジャンルの映画の評価値を一律で1（最低値）に下げた結果、回帰変数値がマイナスになった
Dramaジャンルの映画は一般的に受けが悪いことをここで確認することができる

In [11]:
target_column = 'genre'
data = []
for voc in fm.fields_map[target_column].vocabulary:
    linear_param = fm.fetch_linear_param(column_name=target_column, value=voc)[0]
    target_embedding = fm.fetch_interaction_embedding(column_name=target_column, value=voc)
    
    row = [voc, linear_param]
    for gen in fm.fields_map['gender'].vocabulary:
        emb = fm.fetch_interaction_embedding(column_name='gender', value=gen)
        dot = np.dot(emb, target_embedding)
        row.append(dot)
    data.append(row)

columns = ['genre', 'linear_param'] + fm.fields_map['gender'].vocabulary
pd.DataFrame(data, columns=columns)

,genre,linear_param,M,F
0,Documentary,1.140768,0.201147,0.188036
1,Sci-Fi,0.145037,0.310127,0.136943
2,Thriller,0.359077,0.209874,0.112883
3,Romance,0.461896,0.187126,0.171066
4,Children's,-0.128151,0.327944,0.468894
5,Western,0.497134,0.210614,-0.025274
6,Film-Noir,0.752764,0.224236,0.008550
7,Comedy,0.430605,0.179451,0.126105
8,Fantasy,0.374928,0.253541,0.128044
9,Animation,0.508060,0.314532,0.099622
